<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Hello_Neuron_Your_First_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial 1: Hello Neuron - Your First Agent

In this tutorial, we'll build the foundation of the Neuron framework from scratch and create our very first agent that can receive and respond to messages.

What we'll learn:

 - Basic agent architecture and message passing

 - Threading for concurrent message processing

 - Simple metrics collection

 - Agent lifecycle management (start/stop)

 This forms the core building block for all future tutorials!

In [1]:


import uuid          # For generating unique IDs for agents and messages
import time          # For timestamps and timing operations
import threading     # For concurrent message processing
import queue         # For thread-safe message queues
from abc import ABC, abstractmethod  # For abstract base classes
from dataclasses import dataclass, field  # For clean data structures
from typing import Any, Dict, List, Optional  # For type hints
from enum import Enum  # For enumerated constants

# Define basic types that we'll use throughout the framework
AgentID = str    # Simple string ID for agents
MessageID = str  # Simple string ID for messages

class MessagePriority(Enum):
    """
    Message priority levels - higher numbers = higher priority
    This allows important messages to be processed first
    """
    LOW = 1      # Background tasks, non-urgent information
    NORMAL = 2   # Standard priority for most messages
    HIGH = 3     # Important messages that should be processed quickly
    URGENT = 4   # Critical messages requiring immediate attention

@dataclass
class Message:
    """
    Basic message structure for agent communication

    This is the fundamental unit of communication between agents.
    Every message has a sender, recipients, content, and metadata.
    """
    id: MessageID                                           # Unique identifier for this message
    sender: AgentID                                        # Who sent this message
    recipients: List[AgentID]                              # Who should receive this message
    content: Any                                           # The actual message payload (can be anything)
    priority: MessagePriority = MessagePriority.NORMAL    # How urgent is this message
    metadata: Dict[str, Any] = field(default_factory=dict) # Extra information about the message
    created_at: float = field(default_factory=time.time)   # When was this message created

    @classmethod
    def create(cls, sender: AgentID, recipients: List[AgentID], content: Any,
               priority: MessagePriority = MessagePriority.NORMAL) -> 'Message':
        """
        Factory method to create a new message with auto-generated ID
        This is the preferred way to create messages
        """
        return cls(
            id=str(uuid.uuid4()),  # Generate a unique ID
            sender=sender,
            recipients=recipients,
            content=content,
            priority=priority
        )

class AgentMetrics:
    """
    Simple metrics tracking for agents

    This helps us monitor how our agents are performing:
    - How many messages have they processed?
    - How long does processing take?
    - Are there any errors?
    """
    def __init__(self):
        self.message_count = 0      # Total messages processed
        self.processing_time = 0.0  # Total time spent processing (seconds)
        self.error_count = 0        # Number of errors encountered
        self.last_active = None     # When was the agent last active

    def update_processing_time(self, time_delta: float):
        """Update the total processing time and mark agent as active"""
        self.processing_time += time_delta
        self.last_active = time.time()

    def increment_message_count(self):
        """Increment message count and mark agent as active"""
        self.message_count += 1
        self.last_active = time.time()

class BaseAgent(ABC):
    """
    Base class for all agents in our Neuron framework

    This provides the core functionality that every agent needs:
    - Message queue for receiving messages
    - Threading for concurrent processing
    - Metrics collection
    - Lifecycle management (start/stop)

    All specific agent types will inherit from this class.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = ""):
        # Core identification
        self.id = agent_id or str(uuid.uuid4())  # Generate ID if not provided
        self.name = name or self.__class__.__name__  # Use class name if no name given

        # Message processing infrastructure
        self._message_queue = queue.Queue()  # Thread-safe queue for incoming messages
        self._stop_event = threading.Event()  # Signal to stop processing
        self._processing_thread = None       # Thread that processes messages

        # Monitoring and metrics
        self._metrics = AgentMetrics()  # Track agent performance
        self._running = False           # Is the agent currently running?

        print(f"🤖 Initialized agent: {self.name} ({self.id[:8]}...)")  # Show short ID

    def start(self):
        """
        Start the agent's message processing

        This creates a background thread that continuously processes
        messages from the queue. The agent can receive messages
        even when not started, but won't process them until started.
        """
        if self._running:
            print(f"⚠️  Agent {self.name} is already running")
            return

        # Clear any previous stop signal and start fresh
        self._stop_event.clear()
        self._processing_thread = threading.Thread(
            target=self._processing_loop,
            daemon=True,  # Dies when main program exits
            name=f"Agent-{self.name}"
        )
        self._processing_thread.start()
        self._running = True
        print(f"▶️  Agent {self.name} started")

    def stop(self):
        """
        Stop the agent's message processing

        This gracefully shuts down the agent by signaling the
        processing thread to stop and waiting for it to finish.
        """
        if not self._running:
            return

        # Signal the processing thread to stop
        self._stop_event.set()

        # Wait for the thread to finish (with timeout for safety)
        if self._processing_thread:
            self._processing_thread.join(timeout=2.0)
            if self._processing_thread.is_alive():
                print(f"⚠️  Warning: Agent {self.name} thread didn't stop cleanly")

        self._running = False
        print(f"⏹️  Agent {self.name} stopped")

    def receive_message(self, message: Message):
        """
        Receive a message for processing

        This puts the message in the queue. If the agent is running,
        it will be processed by the background thread.
        """
        self._message_queue.put(message)
        print(f"📬 Agent {self.name} received message: {message.content}")

    def send_message(self, recipients: List[AgentID], content: Any,
                    priority: MessagePriority = MessagePriority.NORMAL) -> Message:
        """
        Send a message to other agents

        In a full system, this would go through a message bus.
        For now, we just create the message and return it.
        """
        message = Message.create(
            sender=self.id,
            recipients=recipients,
            content=content,
            priority=priority
        )

        # Update our metrics
        self._metrics.increment_message_count()
        print(f"📤 Agent {self.name} sent message: {content}")

        return message

    @abstractmethod
    def process_message(self, message: Message):
        """
        Process a received message - must be implemented by subclasses

        This is where each agent type implements its specific logic.
        Every agent must define how it handles incoming messages.
        """
        pass

    def _processing_loop(self):
        """
        Main message processing loop - runs in background thread

        This continuously pulls messages from the queue and processes them.
        It handles errors gracefully and tracks processing metrics.
        """
        print(f"🔄 Agent {self.name} processing loop started")

        while not self._stop_event.is_set():
            try:
                # Try to get a message from the queue (with timeout to check stop signal)
                message = self._message_queue.get(timeout=0.1)

                # Process the message and measure how long it takes
                start_time = time.time()
                self.process_message(message)
                processing_time = time.time() - start_time

                # Update metrics
                self._metrics.update_processing_time(processing_time)

                # Mark the queue task as done
                self._message_queue.task_done()

                print(f"✅ Agent {self.name} processed message in {processing_time:.3f}s")

            except queue.Empty:
                # No messages to process, continue checking for stop signal
                continue
            except Exception as e:
                # Something went wrong processing the message
                print(f"❌ Error processing message in {self.name}: {e}")
                self._metrics.error_count += 1

        print(f"🛑 Agent {self.name} processing loop stopped")

    def get_metrics(self) -> Dict[str, Any]:
        """
        Get agent performance metrics

        This returns a dictionary with all the metrics we've collected
        about this agent's performance.
        """
        return {
            "message_count": self._metrics.message_count,
            "processing_time": self._metrics.processing_time,
            "error_count": self._metrics.error_count,
            "last_active": self._metrics.last_active,
            "average_processing_time": (
                self._metrics.processing_time / max(self._metrics.message_count, 1)
            )
        }

class HelloAgent(BaseAgent):
    """Simple agent that responds with 'Hello World' to any message"""

    def process_message(self, message: Message):
        """Process incoming messages with Hello World response"""
        print(f"Agent {self.name} received message from {message.sender}: {message.content}")

        # Create a hello response
        response = self.send_message(
            recipients=[message.sender],
            content="Hello World! 🌍",
            priority=MessagePriority.NORMAL
        )

        print(f"Agent {self.name} sent response: {response.content}")

class HelloAgent(BaseAgent):
    """
    Simple agent that responds with 'Hello World' to any message

    This is our first concrete agent implementation. It demonstrates
    the basic pattern of receiving a message, processing it, and
    sending a response back to the sender.
    """

    def process_message(self, message: Message):
        """
        Process incoming messages with Hello World response

        This is where our agent's behavior is defined. For this simple
        agent, we just respond with a friendly greeting to any message.
        """
        print(f"🎯 Agent {self.name} processing message from {message.sender}")
        print(f"   Original message: '{message.content}'")

        # Create a friendly response message
        response = self.send_message(
            recipients=[message.sender],  # Send back to whoever sent the original
            content="Hello World! 🌍 Thanks for your message!",
            priority=MessagePriority.NORMAL
        )

        print(f"   Response sent: '{response.content}'")



In [2]:
# =============================================================================
# DEMO SECTION: Let's test our agent!
# =============================================================================

print("=" * 60)
print("🚀 Tutorial 1: Hello Neuron - Your First Agent")
print("=" * 60)
print()

# Step 1: Create our HelloAgent
print("📝 Step 1: Creating a HelloAgent...")
hello_agent = HelloAgent(name="HelloBot")
print()

# Step 2: Start the agent so it can process messages
print("📝 Step 2: Starting the agent...")
hello_agent.start()
print()

# Step 3: Create a test message to send to our agent
print("📝 Step 3: Creating a test message...")
test_message = Message.create(
    sender="user_123",                    # Pretend this is from a user
    recipients=[hello_agent.id],          # Send it to our agent
    content="Hi there! How are you?"      # Friendly test message
)
print(f"   Created message: '{test_message.content}'")
print(f"   Message ID: {test_message.id[:8]}...")  # Show short ID
print()

# Step 4: Send the message to our agent
print("📝 Step 4: Sending message to agent...")
hello_agent.receive_message(test_message)
print()

# Step 5: Wait a moment for the agent to process the message
print("📝 Step 5: Waiting for message processing...")
time.sleep(0.5)  # Give the background thread time to process
print()

# Step 6: Check how our agent performed
print("📝 Step 6: Checking agent metrics...")
metrics = hello_agent.get_metrics()
print("   Agent Performance:")
for key, value in metrics.items():
    if key == 'last_active' and value:
        # Convert timestamp to readable format
        last_active_str = time.strftime('%H:%M:%S', time.localtime(value))
        print(f"     {key}: {last_active_str}")
    elif key == 'processing_time':
        print(f"     {key}: {value:.3f} seconds")
    else:
        print(f"     {key}: {value}")
print()

# Step 7: Clean up by stopping the agent
print("📝 Step 7: Stopping the agent...")
hello_agent.stop()
print()

print("✅ Tutorial 1 Complete!")
print()



🚀 Tutorial 1: Hello Neuron - Your First Agent

📝 Step 1: Creating a HelloAgent...
🤖 Initialized agent: HelloBot (ff55063f...)

📝 Step 2: Starting the agent...
🔄 Agent HelloBot processing loop started
▶️  Agent HelloBot started

📝 Step 3: Creating a test message...
   Created message: 'Hi there! How are you?'
   Message ID: 18a591c4...

📝 Step 4: Sending message to agent...
📬 Agent HelloBot received message: Hi there! How are you?

📝 Step 5: Waiting for message processing...
🎯 Agent HelloBot processing message from user_123
   Original message: 'Hi there! How are you?'
📤 Agent HelloBot sent message: Hello World! 🌍 Thanks for your message!
   Response sent: 'Hello World! 🌍 Thanks for your message!'
✅ Agent HelloBot processed message in 0.000s

📝 Step 6: Checking agent metrics...
   Agent Performance:
     message_count: 1
     processing_time: 0.000 seconds
     error_count: 0
     last_active: 15:52:21
     average_processing_time: 6.29425048828125e-05

📝 Step 7: Stopping the agent...
🛑

In [3]:
# =============================================================================
# SUMMARY OF WHAT WE LEARNED
# =============================================================================

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print("1. 🏗️  Built the foundation of agent architecture")

print("   - Message structure with IDs, priorities, and metadata")

print("   - BaseAgent class with threading and lifecycle management")

print("   - Metrics collection for monitoring agent performance")

print()

print("2. 🔄 Implemented asynchronous message processing")

print("   - Thread-safe message queues")

print("   - Background processing threads")

print("   - Graceful start/stop mechanisms")

print()

print("3. 🤖 Created our first concrete agent")

print("   - HelloAgent that responds to any message")

print("   - Demonstrated message processing pattern")

print("   - Showed how agents communicate")

print()

print("4. 📊 Added monitoring capabilities")

print("   - Processing time measurement")

print("   - Message count tracking")

print("   - Error count tracking")

print()

# =============================================================================
# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")

print("=" * 40)

print("1. 🐛 'Agent thread didn't stop cleanly'")

print("   Problem: Processing loop is stuck or taking too long")

print("   Solution: Check for infinite loops in process_message()")

print("   Solution: Use shorter timeouts in blocking operations")

print()

print("2. 🐛 'queue.Empty' exceptions in console")

print("   Problem: This is normal! It's how we check for stop signals")

print("   Solution: These can be safely ignored")

print()

print("3. 🐛 Messages not being processed")

print("   Problem: Agent not started or thread crashed")

print("   Solution: Always call agent.start() before sending messages")

print("   Solution: Check for exceptions in process_message()")

print()

print("4. 🐛 High memory usage with many messages")

print("   Problem: Message queue growing without processing")

print("   Solution: Ensure process_message() completes quickly")

print("   Solution: Add queue size limits in production")

print()

print("🎉 Ready for Tutorial 2: Memory Basics!")

print("   Next we'll teach our agents to remember things...")

📚 WHAT WE LEARNED:
1. 🏗️  Built the foundation of agent architecture
   - Message structure with IDs, priorities, and metadata
   - BaseAgent class with threading and lifecycle management
   - Metrics collection for monitoring agent performance

2. 🔄 Implemented asynchronous message processing
   - Thread-safe message queues
   - Background processing threads
   - Graceful start/stop mechanisms

3. 🤖 Created our first concrete agent
   - HelloAgent that responds to any message
   - Demonstrated message processing pattern
   - Showed how agents communicate

4. 📊 Added monitoring capabilities
   - Processing time measurement
   - Message count tracking
   - Error count tracking

⚠️  COMMON ERRORS AND SOLUTIONS:
1. 🐛 'Agent thread didn't stop cleanly'
   Problem: Processing loop is stuck or taking too long
   Solution: Check for infinite loops in process_message()
   Solution: Use shorter timeouts in blocking operations

2. 🐛 'queue.Empty' exceptions in console
   Problem: This is normal!